We will account for holidays and weekends in this dataset.

# 0) Import Modules

In [1]:
import os
from zipfile import ZipFile

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

import random
import warnings
warnings.filterwarnings('ignore')


# 1) Load Data

In [2]:
os.chdir("/Users/anhthyngo/Documents/NYU/Fall 2019/DS-GA 1001/project/ds-ga1001/")

In [3]:
cwd = os.getcwd()
datadir = '/'.join(cwd.split('/')) + '/data/'
cal = datadir + 'calendar.csv.zip'
listings = datadir + 'airbnb_clean.csv'
holidays = datadir + 'holidays.txt'

In [4]:
print(listings)
print(cal)

/Users/anhthyngo/Documents/NYU/Fall 2019/DS-GA 1001/project/ds-ga1001/data/airbnb_clean.csv
/Users/anhthyngo/Documents/NYU/Fall 2019/DS-GA 1001/project/ds-ga1001/data/calendar.csv.zip


In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
zb = ZipFile(cal)
cal = pd.read_csv(zb.open("calendar.csv"),low_memory=False)
data = pd.read_csv(listings,low_memory=False)

# 3) Clean Data

In [6]:
cal.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,9668,2019-09-13,f,$51.00,$51.00,3.0,365.0
1,26969,2019-09-13,f,$350.00,$350.00,3.0,33.0
2,26969,2019-09-14,f,$350.00,$350.00,3.0,33.0
3,26969,2019-09-15,f,$350.00,$350.00,3.0,33.0
4,26969,2019-09-16,f,$350.00,$350.00,3.0,33.0


### 3.1 Subset Data

In [7]:
cal = cal[['listing_id','date','price']]

In [8]:
cal.head()

,listing_id,date,price
0,9668,2019-09-13,$51.00
1,26969,2019-09-13,$350.00
2,26969,2019-09-14,$350.00
3,26969,2019-09-15,$350.00
4,26969,2019-09-16,$350.00


### 3.2 Clean Price Data

In [9]:
cal['price'] = cal['price'].str.replace('$', '')
cal['price'] = cal['price'].str.replace(',', '').astype(float)
cal.head()

,listing_id,date,price
0,9668,2019-09-13,51.0
1,26969,2019-09-13,350.0
2,26969,2019-09-14,350.0
3,26969,2019-09-15,350.0
4,26969,2019-09-16,350.0


# 4) Join Listing Data with Calendar Data

In [10]:
data_merge = data.merge(cal, how = 'left', left_on='id', right_on='listing_id')

In [12]:
data.shape

(47455, 230)

In [13]:
data = data_merge[data_merge.columns[~data_merge.columns.isin(['price_x','listing_id'])]]

In [14]:
data.head()

,id,host_response_rate,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,guests_included,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,extra_people,cleaning_fee,security_deposit,host_is_superhost,host_has_profile_pic,host_identity_verified,is_location_exact,has_availability,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification,host_verifications_ facebook,host_verifications_ google,host_verifications_ government_id,host_verifications_ identity_manual,host_verifications_ jumio,host_verifications_ kba,host_verifications_ manual_offline,host_verifications_ manual_online,host_verifications_ offline_government_id,host_verifications_ phone,host_verifications_ reviews,host_verifications_ selfie,host_verifications_ sent_id,host_verifications_ sesame,host_verifications_ sesame_offline,host_verifications_ weibo,host_verifications_ work_email,host_verifications_ zhima_selfie,host_verifications_email,host_verifications_facebook,host_verifications_google,host_verifications_jumio,host_verifications_offline_government_id,host_verifications_phone,host_verifications_reviews,amenities_ toilet,amenities_24-hour check-in,amenities_Accessible-height bed,amenities_Accessible-height toilet,amenities_Air conditioning,amenities_Air purifier,amenities_BBQ grill,amenities_Baby bath,amenities_Baby monitor,amenities_Babysitter recommendations,amenities_Bathtub,amenities_Bathtub with bath chair,amenities_Beach essentials,amenities_Beachfront,amenities_Bed linens,amenities_Breakfast,amenities_Building staff,amenities_Buzzer/wireless intercom,amenities_Cable TV,amenities_Carbon monoxide detector,amenities_Cat(s),amenities_Ceiling hoist,amenities_Changing table,amenities_Children’s books and toys,amenities_Children’s dinnerware,amenities_Cleaning before checkout,amenities_Coffee maker,amenities_Cooking basics,amenities_Crib,amenities_Disabled parking spot,amenities_Dishes and silverware,amenities_Dishwasher,amenities_Dog(s),amenities_Doorman,amenities_Dryer,amenities_EV charger,amenities_Electric profiling bed,amenities_Elevator,amenities_Essentials,amenities_Ethernet connection,amenities_Extra pillows and blankets,amenities_Extra space around bed,amenities_Family/kid friendly,amenities_Fire extinguisher,amenities_Fireplace guards,amenities_Firm mattress,amenities_First aid kit,amenities_Fixed grab bars for shower,amenities_Fixed grab bars for toilet,amenities_Flat path to guest entrance,amenities_Free parking on premises,amenities_Free street parking,amenities_Full kitchen,amenities_Game console,amenities_Garden or backyard,amenities_Ground floor access,amenities_Gym,amenities_Hair dryer,amenities_Handheld shower head,amenities_Hangers,amenities_Heating,amenities_High chair,amenities_Host greets you,amenities_Hot tub,amenities_Hot water,amenities_Hot water kettle,amenities_Indoor fireplace,amenities_Internet,amenities_Iron,amenities_Keypad,amenities_Kitchen,amenities_Kitchenette,amenities_Lake access,amenities_Laptop friendly workspace,amenities_Lock on bedroom door,amenities_Lockbox,amenities_Long term stays allowed,amenities_Luggage dropoff allowed,amenities_Microwave,amenities_Mobile hoist,amenities_No stairs or steps to enter,amenities_Other,amenities_Other pet(s),amenities_Outlet covers,amenities_Oven,amenities_Pack ’n Play/travel crib,amenities_Paid parking off premises,ame

# 5) Join Data with US Bank Holiday Dataset

In [15]:
holiday_df = pd.read_csv(holidays, sep=",", names = ['id','date','holiday'])
holiday_df = holiday_df[['date','holiday']]

In [16]:
holiday_df['date'] = holiday_df['date'].astype('datetime64[ns]').dt.date

### 5.1 Sample Dataset

The dataset is pretty large and keeps killing the kernel.
We will sample 1000 listings.

In [17]:
rand = data['id'].unique()
sample = random.sample(list(rand),1000)

sample_data = data[data['id'].isin(sample)]

In [18]:
sample_data['date'] = sample_data['date'].astype('datetime64[ns]').dt.date

In [19]:
sample_data.shape

(365002, 231)

In [20]:
data_merge = sample_data.merge(holiday_df, how = "left", on = 'date')

In [21]:
data_merge['holiday'] = data_merge['holiday'].fillna("Not A Holiday")

In [22]:
data_merge.head()

,id,host_response_rate,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,guests_included,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,extra_people,cleaning_fee,security_deposit,host_is_superhost,host_has_profile_pic,host_identity_verified,is_location_exact,has_availability,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification,host_verifications_ facebook,host_verifications_ google,host_verifications_ government_id,host_verifications_ identity_manual,host_verifications_ jumio,host_verifications_ kba,host_verifications_ manual_offline,host_verifications_ manual_online,host_verifications_ offline_government_id,host_verifications_ phone,host_verifications_ reviews,host_verifications_ selfie,host_verifications_ sent_id,host_verifications_ sesame,host_verifications_ sesame_offline,host_verifications_ weibo,host_verifications_ work_email,host_verifications_ zhima_selfie,host_verifications_email,host_verifications_facebook,host_verifications_google,host_verifications_jumio,host_verifications_offline_government_id,host_verifications_phone,host_verifications_reviews,amenities_ toilet,amenities_24-hour check-in,amenities_Accessible-height bed,amenities_Accessible-height toilet,amenities_Air conditioning,amenities_Air purifier,amenities_BBQ grill,amenities_Baby bath,amenities_Baby monitor,amenities_Babysitter recommendations,amenities_Bathtub,amenities_Bathtub with bath chair,amenities_Beach essentials,amenities_Beachfront,amenities_Bed linens,amenities_Breakfast,amenities_Building staff,amenities_Buzzer/wireless intercom,amenities_Cable TV,amenities_Carbon monoxide detector,amenities_Cat(s),amenities_Ceiling hoist,amenities_Changing table,amenities_Children’s books and toys,amenities_Children’s dinnerware,amenities_Cleaning before checkout,amenities_Coffee maker,amenities_Cooking basics,amenities_Crib,amenities_Disabled parking spot,amenities_Dishes and silverware,amenities_Dishwasher,amenities_Dog(s),amenities_Doorman,amenities_Dryer,amenities_EV charger,amenities_Electric profiling bed,amenities_Elevator,amenities_Essentials,amenities_Ethernet connection,amenities_Extra pillows and blankets,amenities_Extra space around bed,amenities_Family/kid friendly,amenities_Fire extinguisher,amenities_Fireplace guards,amenities_Firm mattress,amenities_First aid kit,amenities_Fixed grab bars for shower,amenities_Fixed grab bars for toilet,amenities_Flat path to guest entrance,amenities_Free parking on premises,amenities_Free street parking,amenities_Full kitchen,amenities_Game console,amenities_Garden or backyard,amenities_Ground floor access,amenities_Gym,amenities_Hair dryer,amenities_Handheld shower head,amenities_Hangers,amenities_Heating,amenities_High chair,amenities_Host greets you,amenities_Hot tub,amenities_Hot water,amenities_Hot water kettle,amenities_Indoor fireplace,amenities_Internet,amenities_Iron,amenities_Keypad,amenities_Kitchen,amenities_Kitchenette,amenities_Lake access,amenities_Laptop friendly workspace,amenities_Lock on bedroom door,amenities_Lockbox,amenities_Long term stays allowed,amenities_Luggage dropoff allowed,amenities_Microwave,amenities_Mobile hoist,amenities_No stairs or steps to enter,amenities_Other,amenities_Other pet(s),amenities_Outlet covers,amenities_Oven,amenities_Pack ’n Play/travel crib,amenities_Paid parking off premises,ame

In [23]:
data_merge['date'] = data_merge['date'].astype('datetime64[ns]')

In [24]:
data_merge['DayOfWeek'] = data_merge['date'].dt.dayofweek

In [25]:
data_merge.head()

,id,host_response_rate,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,guests_included,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,extra_people,cleaning_fee,security_deposit,host_is_superhost,host_has_profile_pic,host_identity_verified,is_location_exact,has_availability,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification,host_verifications_ facebook,host_verifications_ google,host_verifications_ government_id,host_verifications_ identity_manual,host_verifications_ jumio,host_verifications_ kba,host_verifications_ manual_offline,host_verifications_ manual_online,host_verifications_ offline_government_id,host_verifications_ phone,host_verifications_ reviews,host_verifications_ selfie,host_verifications_ sent_id,host_verifications_ sesame,host_verifications_ sesame_offline,host_verifications_ weibo,host_verifications_ work_email,host_verifications_ zhima_selfie,host_verifications_email,host_verifications_facebook,host_verifications_google,host_verifications_jumio,host_verifications_offline_government_id,host_verifications_phone,host_verifications_reviews,amenities_ toilet,amenities_24-hour check-in,amenities_Accessible-height bed,amenities_Accessible-height toilet,amenities_Air conditioning,amenities_Air purifier,amenities_BBQ grill,amenities_Baby bath,amenities_Baby monitor,amenities_Babysitter recommendations,amenities_Bathtub,amenities_Bathtub with bath chair,amenities_Beach essentials,amenities_Beachfront,amenities_Bed linens,amenities_Breakfast,amenities_Building staff,amenities_Buzzer/wireless intercom,amenities_Cable TV,amenities_Carbon monoxide detector,amenities_Cat(s),amenities_Ceiling hoist,amenities_Changing table,amenities_Children’s books and toys,amenities_Children’s dinnerware,amenities_Cleaning before checkout,amenities_Coffee maker,amenities_Cooking basics,amenities_Crib,amenities_Disabled parking spot,amenities_Dishes and silverware,amenities_Dishwasher,amenities_Dog(s),amenities_Doorman,amenities_Dryer,amenities_EV charger,amenities_Electric profiling bed,amenities_Elevator,amenities_Essentials,amenities_Ethernet connection,amenities_Extra pillows and blankets,amenities_Extra space around bed,amenities_Family/kid friendly,amenities_Fire extinguisher,amenities_Fireplace guards,amenities_Firm mattress,amenities_First aid kit,amenities_Fixed grab bars for shower,amenities_Fixed grab bars for toilet,amenities_Flat path to guest entrance,amenities_Free parking on premises,amenities_Free street parking,amenities_Full kitchen,amenities_Game console,amenities_Garden or backyard,amenities_Ground floor access,amenities_Gym,amenities_Hair dryer,amenities_Handheld shower head,amenities_Hangers,amenities_Heating,amenities_High chair,amenities_Host greets you,amenities_Hot tub,amenities_Hot water,amenities_Hot water kettle,amenities_Indoor fireplace,amenities_Internet,amenities_Iron,amenities_Keypad,amenities_Kitchen,amenities_Kitchenette,amenities_Lake access,amenities_Laptop friendly workspace,amenities_Lock on bedroom door,amenities_Lockbox,amenities_Long term stays allowed,amenities_Luggage dropoff allowed,amenities_Microwave,amenities_Mobile hoist,amenities_No stairs or steps to enter,amenities_Other,amenities_Other pet(s),amenities_Outlet covers,amenities_Oven,amenities_Pack ’n Play/travel crib,amenities_Paid parking off premises,ame

In [26]:
data_merge.shape

(365002, 233)

Create Day of the Week column.

In [27]:
def getDay(x):
    if x == 0:
        return 'Monday'
    elif x == 1:
        return 'Tuesday'
    elif x == 2:
        return 'Wednesday'
    elif x == 3:
        return 'Thursday'
    elif x == 4:
        return 'Friday'
    elif x == 5:
        return 'Saturday'
    else:
        return 'Sunday'

In [28]:
DayOfTheWeek = data_merge['DayOfWeek'].apply(lambda col: getDay(col))
data_merge = data_merge.drop(['DayOfWeek'], axis=1)
data_merge = pd.concat([data_merge,DayOfTheWeek],axis = 1)

In [29]:
data_merge.head()

,id,host_response_rate,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,guests_included,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,extra_people,cleaning_fee,security_deposit,host_is_superhost,host_has_profile_pic,host_identity_verified,is_location_exact,has_availability,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification,host_verifications_ facebook,host_verifications_ google,host_verifications_ government_id,host_verifications_ identity_manual,host_verifications_ jumio,host_verifications_ kba,host_verifications_ manual_offline,host_verifications_ manual_online,host_verifications_ offline_government_id,host_verifications_ phone,host_verifications_ reviews,host_verifications_ selfie,host_verifications_ sent_id,host_verifications_ sesame,host_verifications_ sesame_offline,host_verifications_ weibo,host_verifications_ work_email,host_verifications_ zhima_selfie,host_verifications_email,host_verifications_facebook,host_verifications_google,host_verifications_jumio,host_verifications_offline_government_id,host_verifications_phone,host_verifications_reviews,amenities_ toilet,amenities_24-hour check-in,amenities_Accessible-height bed,amenities_Accessible-height toilet,amenities_Air conditioning,amenities_Air purifier,amenities_BBQ grill,amenities_Baby bath,amenities_Baby monitor,amenities_Babysitter recommendations,amenities_Bathtub,amenities_Bathtub with bath chair,amenities_Beach essentials,amenities_Beachfront,amenities_Bed linens,amenities_Breakfast,amenities_Building staff,amenities_Buzzer/wireless intercom,amenities_Cable TV,amenities_Carbon monoxide detector,amenities_Cat(s),amenities_Ceiling hoist,amenities_Changing table,amenities_Children’s books and toys,amenities_Children’s dinnerware,amenities_Cleaning before checkout,amenities_Coffee maker,amenities_Cooking basics,amenities_Crib,amenities_Disabled parking spot,amenities_Dishes and silverware,amenities_Dishwasher,amenities_Dog(s),amenities_Doorman,amenities_Dryer,amenities_EV charger,amenities_Electric profiling bed,amenities_Elevator,amenities_Essentials,amenities_Ethernet connection,amenities_Extra pillows and blankets,amenities_Extra space around bed,amenities_Family/kid friendly,amenities_Fire extinguisher,amenities_Fireplace guards,amenities_Firm mattress,amenities_First aid kit,amenities_Fixed grab bars for shower,amenities_Fixed grab bars for toilet,amenities_Flat path to guest entrance,amenities_Free parking on premises,amenities_Free street parking,amenities_Full kitchen,amenities_Game console,amenities_Garden or backyard,amenities_Ground floor access,amenities_Gym,amenities_Hair dryer,amenities_Handheld shower head,amenities_Hangers,amenities_Heating,amenities_High chair,amenities_Host greets you,amenities_Hot tub,amenities_Hot water,amenities_Hot water kettle,amenities_Indoor fireplace,amenities_Internet,amenities_Iron,amenities_Keypad,amenities_Kitchen,amenities_Kitchenette,amenities_Lake access,amenities_Laptop friendly workspace,amenities_Lock on bedroom door,amenities_Lockbox,amenities_Long term stays allowed,amenities_Luggage dropoff allowed,amenities_Microwave,amenities_Mobile hoist,amenities_No stairs or steps to enter,amenities_Other,amenities_Other pet(s),amenities_Outlet covers,amenities_Oven,amenities_Pack ’n Play/travel crib,amenities_Paid parking off premises,ame

# 6) Create Dummies for Holidays and Weekends

In [30]:
holiday_dummies = pd.get_dummies(data_merge['holiday'])
data_merge = data_merge.loc[:, data_merge.columns != 'holiday']

In [31]:
holiday_dummies.head()

,Christmas Day,Columbus Day,Independence Day,Labor Day,Martin Luther King Jr. Day,Memorial Day,New Year Day,Not A Holiday,Presidents Day (Washingtons Birthday),Thanksgiving Day,Veterans Day
0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0


In [32]:
data_final = pd.concat([data_merge, holiday_dummies],axis = 1)


In [33]:
data_final.head()

,id,host_response_rate,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,guests_included,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,extra_people,cleaning_fee,security_deposit,host_is_superhost,host_has_profile_pic,host_identity_verified,is_location_exact,has_availability,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification,host_verifications_ facebook,host_verifications_ google,host_verifications_ government_id,host_verifications_ identity_manual,host_verifications_ jumio,host_verifications_ kba,host_verifications_ manual_offline,host_verifications_ manual_online,host_verifications_ offline_government_id,host_verifications_ phone,host_verifications_ reviews,host_verifications_ selfie,host_verifications_ sent_id,host_verifications_ sesame,host_verifications_ sesame_offline,host_verifications_ weibo,host_verifications_ work_email,host_verifications_ zhima_selfie,host_verifications_email,host_verifications_facebook,host_verifications_google,host_verifications_jumio,host_verifications_offline_government_id,host_verifications_phone,host_verifications_reviews,amenities_ toilet,amenities_24-hour check-in,amenities_Accessible-height bed,amenities_Accessible-height toilet,amenities_Air conditioning,amenities_Air purifier,amenities_BBQ grill,amenities_Baby bath,amenities_Baby monitor,amenities_Babysitter recommendations,amenities_Bathtub,amenities_Bathtub with bath chair,amenities_Beach essentials,amenities_Beachfront,amenities_Bed linens,amenities_Breakfast,amenities_Building staff,amenities_Buzzer/wireless intercom,amenities_Cable TV,amenities_Carbon monoxide detector,amenities_Cat(s),amenities_Ceiling hoist,amenities_Changing table,amenities_Children’s books and toys,amenities_Children’s dinnerware,amenities_Cleaning before checkout,amenities_Coffee maker,amenities_Cooking basics,amenities_Crib,amenities_Disabled parking spot,amenities_Dishes and silverware,amenities_Dishwasher,amenities_Dog(s),amenities_Doorman,amenities_Dryer,amenities_EV charger,amenities_Electric profiling bed,amenities_Elevator,amenities_Essentials,amenities_Ethernet connection,amenities_Extra pillows and blankets,amenities_Extra space around bed,amenities_Family/kid friendly,amenities_Fire extinguisher,amenities_Fireplace guards,amenities_Firm mattress,amenities_First aid kit,amenities_Fixed grab bars for shower,amenities_Fixed grab bars for toilet,amenities_Flat path to guest entrance,amenities_Free parking on premises,amenities_Free street parking,amenities_Full kitchen,amenities_Game console,amenities_Garden or backyard,amenities_Ground floor access,amenities_Gym,amenities_Hair dryer,amenities_Handheld shower head,amenities_Hangers,amenities_Heating,amenities_High chair,amenities_Host greets you,amenities_Hot tub,amenities_Hot water,amenities_Hot water kettle,amenities_Indoor fireplace,amenities_Internet,amenities_Iron,amenities_Keypad,amenities_Kitchen,amenities_Kitchenette,amenities_Lake access,amenities_Laptop friendly workspace,amenities_Lock on bedroom door,amenities_Lockbox,amenities_Long term stays allowed,amenities_Luggage dropoff allowed,amenities_Microwave,amenities_Mobile hoist,amenities_No stairs or steps to enter,amenities_Other,amenities_Other pet(s),amenities_Outlet covers,amenities_Oven,amenities_Pack ’n Play/travel crib,amenities_Paid parking off premises,ame

In [34]:
day_dummies = pd.get_dummies(data_merge['DayOfWeek'])
day_dummies.head()

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,1,0,0,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0
3,0,1,0,0,0,0,0
4,0,0,0,0,0,1,0


In [38]:
day_dummies = pd.get_dummies(data_merge['DayOfWeek'])
nodates = data_final.loc[:, data_final.columns != 'DayOfWeek']
data_final = pd.concat([nodates, day_dummies],axis = 1)


In [39]:
data_final.head()

,id,host_response_rate,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,guests_included,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,extra_people,cleaning_fee,security_deposit,host_is_superhost,host_has_profile_pic,host_identity_verified,is_location_exact,has_availability,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification,host_verifications_ facebook,host_verifications_ google,host_verifications_ government_id,host_verifications_ identity_manual,host_verifications_ jumio,host_verifications_ kba,host_verifications_ manual_offline,host_verifications_ manual_online,host_verifications_ offline_government_id,host_verifications_ phone,host_verifications_ reviews,host_verifications_ selfie,host_verifications_ sent_id,host_verifications_ sesame,host_verifications_ sesame_offline,host_verifications_ weibo,host_verifications_ work_email,host_verifications_ zhima_selfie,host_verifications_email,host_verifications_facebook,host_verifications_google,host_verifications_jumio,host_verifications_offline_government_id,host_verifications_phone,host_verifications_reviews,amenities_ toilet,amenities_24-hour check-in,amenities_Accessible-height bed,amenities_Accessible-height toilet,amenities_Air conditioning,amenities_Air purifier,amenities_BBQ grill,amenities_Baby bath,amenities_Baby monitor,amenities_Babysitter recommendations,amenities_Bathtub,amenities_Bathtub with bath chair,amenities_Beach essentials,amenities_Beachfront,amenities_Bed linens,amenities_Breakfast,amenities_Building staff,amenities_Buzzer/wireless intercom,amenities_Cable TV,amenities_Carbon monoxide detector,amenities_Cat(s),amenities_Ceiling hoist,amenities_Changing table,amenities_Children’s books and toys,amenities_Children’s dinnerware,amenities_Cleaning before checkout,amenities_Coffee maker,amenities_Cooking basics,amenities_Crib,amenities_Disabled parking spot,amenities_Dishes and silverware,amenities_Dishwasher,amenities_Dog(s),amenities_Doorman,amenities_Dryer,amenities_EV charger,amenities_Electric profiling bed,amenities_Elevator,amenities_Essentials,amenities_Ethernet connection,amenities_Extra pillows and blankets,amenities_Extra space around bed,amenities_Family/kid friendly,amenities_Fire extinguisher,amenities_Fireplace guards,amenities_Firm mattress,amenities_First aid kit,amenities_Fixed grab bars for shower,amenities_Fixed grab bars for toilet,amenities_Flat path to guest entrance,amenities_Free parking on premises,amenities_Free street parking,amenities_Full kitchen,amenities_Game console,amenities_Garden or backyard,amenities_Ground floor access,amenities_Gym,amenities_Hair dryer,amenities_Handheld shower head,amenities_Hangers,amenities_Heating,amenities_High chair,amenities_Host greets you,amenities_Hot tub,amenities_Hot water,amenities_Hot water kettle,amenities_Indoor fireplace,amenities_Internet,amenities_Iron,amenities_Keypad,amenities_Kitchen,amenities_Kitchenette,amenities_Lake access,amenities_Laptop friendly workspace,amenities_Lock on bedroom door,amenities_Lockbox,amenities_Long term stays allowed,amenities_Luggage dropoff allowed,amenities_Microwave,amenities_Mobile hoist,amenities_No stairs or steps to enter,amenities_Other,amenities_Other pet(s),amenities_Outlet covers,amenities_Oven,amenities_Pack ’n Play/travel crib,amenities_Paid parking off premises,ame

In [40]:
data_final = data_final.drop(['id', 'date'], axis=1)

# 7) Output Dataset

In [41]:
#data_final.to_csv(os.getcwd()+'/data/airbnb_final.csv', index=False)
data_final.to_csv(os.getcwd()+'/data/airbnb_final.csv.gz',index = False, compression='gzip')

### Done!